In [1]:
# Import Libraries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime
import time
import smtplib
import csv
import openpyxl

In [2]:
# Function to fetch the data based on the page number
def extract_raw_data(page_number):
    URL = 'https://www.zillow.com/homes/for_rent/Brooklyn,-New-York,-NY_rb/{}_p/'

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36",
               "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
               "Accept-Encoding":"gzip, deflate, br","upgrade-insecure-requests":"1"}

    page = requests.get(URL.format(page_number), headers=headers)

    soup = BeautifulSoup(page.content, "html.parser")
    
    return soup

In [ ]:
# Getting the property data

# Simplified data
simple_data = []

# Get data from page to page
for page_number in range(20):
    
    # Giving interlude so requests won't be constant and blocked
    time.sleep(10)
    soup = extract_raw_data(page_number)
    
    data = soup.find_all('div',{"class":"StyledPropertyCardDataWrapper-c11n-8-89-0__sc-1omp4c3-0 eBcgiS property-card-data"})
    
    for i in range(len(data)):
        obj = {}
        try:
            obj["address"] = data[i].find("a", {"class":"StyledPropertyCardDataArea-c11n-8-89-0__sc-yipmu-0 gZUDVm property-card-link"}).text
        except:
            obj["address"] = None

        try:
            obj["price"] = data[i].find("div", {"class":"StyledPropertyCardDataArea-c11n-8-89-0__sc-yipmu-0 kvldQr"}).text
        except:
            obj["price"] = None

        try:
            obj["properties"] = data[i].find("div", {"class":"StyledPropertyCardDataArea-c11n-8-89-0__sc-yipmu-0 eLdkcJ"}).text
        except:
            obj["properties"] = None

        simple_data.append(obj)


In [ ]:
# Check if the data is scrape successfully

print(len(simple_data))
simple_data

In [ ]:
# Saving simplified data to xlsx to be processed

# Convert array to dataframe
df = pd.DataFrame(simple_data)

# Save DataFrame to Excel file
df.to_excel('../data/data.xlsx', index=False)